In [1]:
import math
from tqdm import tqdm
import numpy as np
import glob
import csv
import cv2
from PIL import Image

**Scale down** the road data to form an image of roads with a resolution of around 50m x 50m per pixel  
The resulting image is of 656x656 dimentions

In [98]:
print("Data loading and Management")
ml_road_dir = "../ml_preds_csv/"

base_file_name = "3001120103"

# 82*8 = 656
# 164*8 = 1312
res = 656
scale = 100
grid = np.zeros((res, res), dtype=np.uint8)
road_coords = dict()

# level_a = {'0': (0, 0), '1': (0, 328), '2': (328, 0), '3': (328, 328)}
level_a = {'0': (0, 0), '1': (0, int(res/2)), '2': (int(res/2), 0), '3': (int(res/2), int(res/2))}
# level_b = {'0': (0, 0), '1': (0, 164), '2': (164, 0), '3': (164, 164)}
level_b = {'0': (0, 0), '1': (0, int(res/4)), '2': (int(res/4), 0), '3': (int(res/4), int(res/4))}
# level_c = {'0': (0, 0), '1': (0, 82), '2': (82, 0), '3': (82, 82)}
level_c = {'0': (0, 0), '1': (0, int(res/8)), '2': (int(res/8), 0), '3': (int(res/8), int(res/8))}
for a in tqdm(level_a):
    a_i = level_a[a][0]
    a_j = level_a[a][1]
    for b in level_b:
        b_i = a_i + level_b[b][0]
        b_j = a_j + level_b[b][1]
        for c in level_c:
            c_i = b_i + level_c[c][0]
            c_j = b_j + level_c[c][1]

            # generate files names
            file_name = base_file_name + a + b + c + ".csv"
            file_path = ml_road_dir + file_name

            with open(file_path, 'r') as csv_file:
                reader = csv.reader(csv_file)
                first_row = True
                for row in reader:
                    if first_row:
                        first_row = False
                        continue

                    d_i = int(row[0])
                    d_j = int(row[1])

                    i = c_i + int(d_i / scale)
                    j = c_j + int(d_j / scale)

                    grid[i][j] = max(np.uint8(row[2]), grid[i][j])
                    if int(row[2]) >= 75 and (i*res+j not in road_coords or road_coords[i*res+j][0] < int(row[2])):
                            road_coords[i*res+j] = (int(row[2]), float(row[3]), float(row[4]))
                            





  0%|          | 0/4 [00:00<?, ?it/s]

Data loading and Management






 25%|██▌       | 1/4 [01:59<05:57, 119.22s/it]



 50%|█████     | 2/4 [04:12<04:07, 123.54s/it]



 75%|███████▌  | 3/4 [05:57<01:57, 117.77s/it]



100%|██████████| 4/4 [08:47<00:00, 131.76s/it]


In [99]:
grid[grid < 75] = 0
img = Image.fromarray(grid)
img.show()
img.save("road_image.png", "png")
print("Image Size:", np.shape(grid))
print("total pixels: ", res*res)
print("road pixels:", np.count_nonzero(grid))

Image Size: (656, 656)
total pixels:  430336
road pixels: 105923


In [91]:
roads = grid.copy()
roads[roads >= 75] = 1
print("Image Size:", np.shape(roads))
print("total pizels: ", res*res)
print("road pixels:", np.count_nonzero(roads))

# visited = np.zeros(np.shape(roads))
# print("Image Size:", np.shape(visited))
# print("total pizels: ", 656*656)
# print("road pixels:", np.count_nonzero(visited))

Image Size: (6560, 6560)
total pizels:  43033600
road pixels: 1655586


In [92]:
# load test cases
test_cases = []
with open("../routing_challenge_pairs.csv", 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        test_cases.append(row)
        
del test_cases[0]

In [93]:
# find the coordinate (i,j) of a lat/long pair in the scaled down image
def closest_ij(lati, lngi):
    min_dist = 999999
    i=0
    j=0
    lati = float(lati)*100000
    lngi = float(lngi)*100000
    for coord in road_coords:
        lat = float(road_coords[coord][1])*100000
        lng = float(road_coords[coord][2])*100000
        if min_dist > math.sqrt((lati - lat)**2 + (lngi - lng)**2):
            min_dist = math.sqrt((lati - lat)**2 + (lngi - lng)**2)
            i = int(coord/res)
            j = int(coord - i*res)
    return i,j

In [94]:
# An example
lat1 = test_cases[0][0]
lng1 = test_cases[0][1]
i1,j1 = closest_ij(lat1, lng1)
print("1:",(i1,j1))
print("1:", grid[i1][j1])

lat2 = test_cases[0][2]
lng2 = test_cases[0][3]
i2,j2 = closest_ij(lat2, lng2)
print("2:",(i2, j2))
print("2:", grid[i2][j2])

1: (3915, 4574)
1: 214
2: (5622, 4184)
2: 244


Apply **Breath First Search (BFS)** to calculate the minimum number of road pixels between the two image pixels  
If no path is found, return -1

In [95]:
def BFS(i1, j1, i2, j2):
    visited = np.zeros(np.shape(roads))
    queue = []
    queue.append((0, i1, j1))
    visited[i1][j1] = 1
    front = 0
    kernel = [-1,0,1]
    while front != len(queue):
        (d, i, j) = queue[front]
        front += 1
        if i==i2 and j==j2:
            return d
        
        for p in kernel:
            for q in kernel:
                if 0<= i+p < res and 0<= j+q < res and roads[i+p][j+q] != 0 and visited[i+p][j+q] == 0:
                    queue.append((d+1, i+p, j+q))
                    visited[i+p][j+q] = 1
                    
    return -1

In [96]:
# test a sample
dist = BFS(3915, 4574, 5622, 4184)
dist*10

-10

In [97]:
expected = 0
for degree in range(5, 46, 5):
    rad1 = degree * math.pi / 180
    rad2 = (degree-5) * math.pi / 180
    expected += 2*(1/math.cos(rad1))*(math.sin(rad1) - math.sin(rad2))
expected

1.6028889820468648

In [81]:
output_file = open("../results/results_part2_10", 'w')
csv_writer = csv.writer(output_file)
csv_writer.writerow(["latitude_src", "longitude_src", "latitude_dst", "longitude_dst", "distance"])
num_faults = 0
for case in tqdm(test_cases):
    lat1 = case[0]
    lng1 = case[1]
    i1,j1 = closest_ij(lat1, lng1)

    lat2 = case[2]
    lng2 = case[3]
    i2,j2 = closest_ij(lat2, lng2)
    
    dist = BFS(i1, j1, i2, j2)
    
    if dist != -1:
        dist = dist*scale/100000
    else:
        num_faults += 1
        
    csv_writer.writerow([lat1, lng1, lat2, lng2, str(dist)])
output_file.close()
print("Number of faults:", num_faults)




  0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 1/1000 [00:02<33:41,  2.02s/it]


  0%|          | 2/1000 [00:03<30:29,  1.83s/it]


  0%|          | 3/1000 [00:03<24:04,  1.45s/it]


  0%|          | 4/1000 [00:05<23:25,  1.41s/it]


  0%|          | 5/1000 [00:06<22:49,  1.38s/it]


  1%|          | 6/1000 [00:10<33:04,  2.00s/it]


  1%|          | 7/1000 [00:10<24:22,  1.47s/it]


  1%|          | 8/1000 [00:12<29:43,  1.80s/it]


  1%|          | 9/1000 [00:15<32:06,  1.94s/it]


  1%|          | 10/1000 [00:17<36:01,  2.18s/it]


  1%|          | 11/1000 [00:18<26:29,  1.61s/it]


  1%|          | 12/1000 [00:19<25:03,  1.52s/it]


  1%|▏         | 13/1000 [00:22<30:23,  1.85s/it]


  1%|▏         | 14/1000 [00:24<31:13,  1.90s/it]


  2%|▏         | 15/1000 [00:25<27:38,  1.68s/it]


  2%|▏         | 16/1000 [00:26<23:43,  1.45s/it]


  2%|▏         | 17/1000 [00:28<26:04,  1.59s/it]


  2%|▏         | 18/1000 [00:30<32:15,  1.97s/it]


  2%|▏         | 19/1000 [0

 13%|█▎        | 134/1000 [04:21<29:37,  2.05s/it]


 14%|█▎        | 135/1000 [04:24<34:08,  2.37s/it]


 14%|█▎        | 136/1000 [04:25<28:34,  1.98s/it]


 14%|█▎        | 137/1000 [04:29<34:10,  2.38s/it]


 14%|█▍        | 138/1000 [04:30<27:58,  1.95s/it]


 14%|█▍        | 139/1000 [04:31<25:54,  1.81s/it]


 14%|█▍        | 140/1000 [04:35<33:43,  2.35s/it]


 14%|█▍        | 141/1000 [04:38<36:44,  2.57s/it]


 14%|█▍        | 142/1000 [04:41<37:28,  2.62s/it]


 14%|█▍        | 143/1000 [04:43<36:31,  2.56s/it]


 14%|█▍        | 144/1000 [04:44<29:48,  2.09s/it]


 14%|█▍        | 145/1000 [04:46<28:06,  1.97s/it]


 15%|█▍        | 146/1000 [04:46<21:29,  1.51s/it]


 15%|█▍        | 147/1000 [04:48<22:25,  1.58s/it]


 15%|█▍        | 148/1000 [04:49<19:48,  1.40s/it]


 15%|█▍        | 149/1000 [04:49<16:32,  1.17s/it]


 15%|█▌        | 150/1000 [04:50<13:58,  1.01it/s]


 15%|█▌        | 151/1000 [04:53<21:25,  1.51s/it]


 15%|█▌        | 152/1000 [04:57<32:36,  2.31s

 40%|███▉      | 398/1000 [12:56<23:29,  2.34s/it]


 40%|███▉      | 399/1000 [12:56<17:13,  1.72s/it]


 40%|████      | 400/1000 [13:00<22:51,  2.29s/it]


 40%|████      | 401/1000 [13:01<18:57,  1.90s/it]


 40%|████      | 402/1000 [13:02<15:11,  1.52s/it]


 40%|████      | 403/1000 [13:03<13:39,  1.37s/it]


 40%|████      | 404/1000 [13:03<11:06,  1.12s/it]


 40%|████      | 405/1000 [13:04<09:09,  1.08it/s]


 41%|████      | 406/1000 [13:04<08:15,  1.20it/s]


 41%|████      | 407/1000 [13:08<17:20,  1.76s/it]


 41%|████      | 408/1000 [13:10<17:57,  1.82s/it]


 41%|████      | 409/1000 [13:11<16:16,  1.65s/it]


 41%|████      | 410/1000 [13:14<20:37,  2.10s/it]


 41%|████      | 411/1000 [13:17<21:09,  2.15s/it]


 41%|████      | 412/1000 [13:18<17:06,  1.75s/it]


 41%|████▏     | 413/1000 [13:20<17:54,  1.83s/it]


 41%|████▏     | 414/1000 [13:21<18:02,  1.85s/it]


 42%|████▏     | 415/1000 [13:25<21:27,  2.20s/it]


 42%|████▏     | 416/1000 [13:28<25:29,  2.62s

 66%|██████▌   | 662/1000 [21:09<09:01,  1.60s/it]


 66%|██████▋   | 663/1000 [21:10<07:55,  1.41s/it]


 66%|██████▋   | 664/1000 [21:11<07:44,  1.38s/it]


 66%|██████▋   | 665/1000 [21:13<08:46,  1.57s/it]


 67%|██████▋   | 666/1000 [21:15<09:22,  1.68s/it]


 67%|██████▋   | 667/1000 [21:16<09:02,  1.63s/it]


 67%|██████▋   | 668/1000 [21:18<08:10,  1.48s/it]


 67%|██████▋   | 669/1000 [21:19<07:28,  1.35s/it]


 67%|██████▋   | 670/1000 [21:20<07:45,  1.41s/it]


 67%|██████▋   | 671/1000 [21:23<09:49,  1.79s/it]


 67%|██████▋   | 672/1000 [21:25<10:04,  1.84s/it]


 67%|██████▋   | 673/1000 [21:28<12:45,  2.34s/it]


 67%|██████▋   | 674/1000 [21:32<14:31,  2.67s/it]


 68%|██████▊   | 675/1000 [21:32<11:11,  2.07s/it]


 68%|██████▊   | 676/1000 [21:34<10:47,  2.00s/it]


 68%|██████▊   | 677/1000 [21:35<08:01,  1.49s/it]


 68%|██████▊   | 678/1000 [21:37<09:05,  1.70s/it]


 68%|██████▊   | 679/1000 [21:40<11:17,  2.11s/it]


 68%|██████▊   | 680/1000 [21:43<13:22,  2.51s

 93%|█████████▎| 926/1000 [34:11<03:54,  3.17s/it]


 93%|█████████▎| 927/1000 [34:17<04:56,  4.07s/it]


 93%|█████████▎| 928/1000 [34:20<04:30,  3.75s/it]


 93%|█████████▎| 929/1000 [34:21<03:34,  3.02s/it]


 93%|█████████▎| 930/1000 [34:22<02:38,  2.26s/it]


 93%|█████████▎| 931/1000 [34:28<04:04,  3.55s/it]


 93%|█████████▎| 932/1000 [34:30<03:21,  2.96s/it]


 93%|█████████▎| 933/1000 [34:35<04:04,  3.65s/it]


 93%|█████████▎| 934/1000 [34:38<03:56,  3.58s/it]


 94%|█████████▎| 935/1000 [34:42<03:43,  3.44s/it]


 94%|█████████▎| 936/1000 [34:42<02:48,  2.64s/it]


 94%|█████████▎| 937/1000 [34:49<03:57,  3.77s/it]


 94%|█████████▍| 938/1000 [34:51<03:28,  3.36s/it]


 94%|█████████▍| 939/1000 [34:54<03:10,  3.12s/it]


 94%|█████████▍| 940/1000 [34:55<02:42,  2.72s/it]


 94%|█████████▍| 941/1000 [35:01<03:22,  3.43s/it]


 94%|█████████▍| 942/1000 [35:02<02:48,  2.91s/it]


 94%|█████████▍| 943/1000 [35:06<03:02,  3.20s/it]


 94%|█████████▍| 944/1000 [35:08<02:39,  2.85s

Number of faults: 100


In [86]:
6560/2

3280/2

1640.0